<a href="https://colab.research.google.com/github/ucfilho/Raianars_SelfTest_comp_02_July_2020/blob/master/Stats_2000_Shubert3_ALL_D_10_JUL_03_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyade-python

In [2]:
!pip install PyFDE

In [3]:
import pyade.ilshade 
import pyade.sade 
import pyade.jso
import pyade.shade
import pyade.jade 
import pyfde
import numpy as np
import pandas as pd
import time

In [4]:
!git clone https://github.com/ucfilho/Raianars_July_2020 #clonar do Github
%cd Raianars_July_2020

fatal: destination path 'Raianars_July_2020' already exists and is not an empty directory.
/content/Raianars_July_2020


In [5]:
import Go2Ann
import de_soma_Insight
import de_bounds_matriz
from AvaliaX import AvaliaX
from GeraIndices import GeraIndices

In [6]:
import Function
import Go2MinMax

In [7]:
ITE=2000 #10 # PONTOS quantas vezes vai alteral NPAR 
NPAR=100 # NUMBER OF PARTICLES
dim=10
RUNS=50
fchoice='Shubert3' 
Fun=Function.Shubert3

MIN,MAX=Go2MinMax.Intervalo(dim,fchoice)
bounds = [(0,0)]*dim

for i in range(dim):
  bounds[i] = (MIN[i],MAX[i])

In [8]:
############################### INPUT INFORMATION (FRANNK)
Fc=0.1 # fator de correcao parametros da ANN
its=1 #10 # quantas vezes vai usar o NPAR antes de alterar de novo
mut=0.5
crossp=0.5
Fi=mut;CRi=crossp;# para impressão
PAR=len(MAX) #NUM DE PARAMETROS A SER OTIMIZADOS

In [9]:
# JADE
start_time = time.time()
algorithm = pyade.jso
BEST_AGENTS=[]
for i in range(RUNS):
  ANSWER_AGENTS=[]
  solver = pyfde.JADE(Fun, n_dim=dim, n_pop=NPAR, limits=bounds, minimize=True)
  solver.p = 0.05
  solver.c = 0.1
  solver.cr = 0.5
  solver.f = 0.5
  best, fit = solver.run(n_it=ITE)
  fit=(-1)*fit
  for x in best:
    ANSWER_AGENTS.append(x)
  ANSWER_AGENTS.append(fit)
  BEST_AGENTS.append(ANSWER_AGENTS)
BEST_AGENTS=np.array(BEST_AGENTS)
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()
nome='JADE_Fo-'+str(0)+'_CRo-'+str(0)
print(nome,'NPAR',NPAR,'Iterações=',ITE,'Dim=',dim ,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print( '\n')
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

JADE_Fo-0_CRo-0 NPAR 100 Iterações= 2000 Dim= 10 Min/Max= -10 / 10 _ Shubert3


MÉDIA:  -148.379500257106
MIN __: -148.37950025710603
MAX__:  -148.379500257106
STD __: 1.6077746776921856e-14
Tempo Médio= 18.488 segundos ---
50 Rodadas

BEST:  [  -7.39728499   -1.11409969   -1.11409969    5.16908562   -1.11409968
    5.16908562    5.16908562   -7.397285     -7.39728499    5.16908562
 -148.37950026]


In [10]:
# FRANNK START HERE !!!!!

In [11]:
!git clone https://github.com/ucfilho/ANN_read_matrix #clonar do Github
%cd ANN_read_matrix

fatal: destination path 'ANN_read_matrix' already exists and is not an empty directory.
/content/Raianars_July_2020/ANN_read_matrix


In [12]:
ANN_F=pd.read_csv('ANN_INSIGHT_Valor_F.csv')
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')
syn0_F,syn1_F,X_max_F,X_min_F=Go2Ann.GetMatriz(ANN_F)
syn0_CR,syn1_CR,X_max_CR,X_min_CR=Go2Ann.GetMatriz(ANN_CR)
fields = ['DI RELATIVO','FRAC Its','Fo','CRo']

#TOTAL=7*dim*its*ITE
TOTAL=10*dim*its*ITE
TOTAL=int(TOTAL/2)   # CONFERIR SE ESTE EH O MELHOR LUGAR

dimensions=len(MAX)
start_time = time.time() #INICIALIZA CONTAGEM DO TEMPO

MAX_P=np.zeros(ITE) # ATENCAO para variar F e Cr MAX_P passa a ser 3 vezes maior
MIN_P=np.zeros(ITE) # ATENCAO para variar F e Cr MIN_P passa a ser 3 vezes maior

for i in range(dim):
  MAX_P[i]=MAX[i] 
  MIN_P[i]=MIN[i]

rodadas=1
for i in range(rodadas):
  SOMA=0
  pop = np.random.rand(NPAR, dimensions)
  diff=np.zeros(PAR)
  for ii in range(PAR):
    diff[ii] = MAX[ii]-MIN[ii]
  X = MIN + pop * diff
    
  PONTOS=ITE # DIVISOES DA POPULACAO
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]
for i in range(RUNS):
    MAT=[] 
    mut=Fi
    crossp=CRi
    SOMA=0
    SOMAf=0
    start_ite = time.time()
    delta_time=(start_ite-start_time)/60
    print('ite=',i,'total=50','time=',delta_time)
    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(PAR)
    for ii in range(PAR):
      diff[ii] = MAX[ii]-MIN[ii]
    X = MIN + pop * diff
    Xf = MIN + pop * diff
    
    DIo=1e99
  
    CRo=crossp;Fo=mut;
    Fitness = np.asarray([Fun(ind) for ind in X])

    XY,BEST_XY,BESTo,FOBESTo=AvaliaX(X,Fitness)
    nrow,ncol=X.shape

    QUANT=17;MAT_INDo=np.zeros((1,QUANT)); # quantos indices esta fazendo
    MAT_INDo[0,5]=nrow  # VALOR NP
    MAT_INDo[0,6]=Fo    # VALOR Fo   
    MAT_INDo[0,7]=CRo   # VALOR CRo
  
    MAT_INDo=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                        syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                        X_min_CR,Fc,fields,Fun)
  
    DIo=np.copy(MAT_INDo[0,0])
    MAT_INDo[0,15]=Fo    # VALOR Fo   
    MAT_INDo[0,16]=CRo   # VALOR CRo
    #ind_crossp=ITE-1

    
    for k in range(ITE):
      X,BEST,FOBEST,XY,BEST_XY,SOMA= de_soma_Insight.de(MAX,MIN, mut, crossp,NPAR,its,Fun,X,SOMA,TOTAL)
      #Xf,BESTf,FOBESTf,XYf,BEST_XYf,SOMAf= de_soma_Insight.de(MAX,MIN, Fi, CRi, NPAR,its,Fun,Xf,SOMAf,TOTAL)

      
      MAT_IND=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                          syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                          X_min_CR,Fc,fields,Fun)
    
      BESTo=np.copy(BEST);FOBESTo=np.copy(FOBEST);
      MAT.append(MAT_IND.ravel())
    
      MAT_INDo[0,6]=MAT_IND[0,15]   # VALOR Fo   
      MAT_INDo[0,7]=MAT_IND[0,16]   # VALOR CRo
      mut=MAT_IND[0,15]
      crossp=MAT_IND[0,16]

      Fitness = np.asarray([Fun(ind) for ind in X])
      #Fitnessf = np.asarray([Fun(ind) for ind in Xf])
      Maior=np.argmax(Fitness)
      #Maiorf=np.argmax(Fitnessf)
      Menor=np.argmin(Fitness)
      #Menorf=np.argmin(Fitnessf)

      #X[Maior,:]=np.copy(Xf[Menorf,:])
      #Xf[Maiorf,:]=np.copy(X[Menor,:])
      
    BEST_AGENTS.append(BEST_XY)
    MAT_stat.append(MAT)
    #print("GlobalParam e Solucao:", BEST,"Funcao=",FOBEST)
    
############################## Cálculo dos Parâmetros de Avaliação   
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()

nome='_ANN_Div3x4_Fo-'+str(Fi)+'_CRo-'+str(CRi)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('DE_POP_VAR_F_CR','TOTAL=',TOTAL,'Iterações=',ITE*its,'Dim=', PAR,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

ite= 0 total=50 time= 7.772445678710938e-06
ite= 1 total=50 time= 2.9363307873408
ite= 2 total=50 time= 5.879115664958954
ite= 3 total=50 time= 8.846981823444366
ite= 4 total=50 time= 11.802908710638683
ite= 5 total=50 time= 14.735221179326375
ite= 6 total=50 time= 17.670075007279713
ite= 7 total=50 time= 20.619025802612306
ite= 8 total=50 time= 23.559594949086506
ite= 9 total=50 time= 26.498581083615623
ite= 10 total=50 time= 29.444395351409913
ite= 11 total=50 time= 32.394142524401346
ite= 12 total=50 time= 35.33949728012085
ite= 13 total=50 time= 38.28399724165599
ite= 14 total=50 time= 41.23733492294947
ite= 15 total=50 time= 44.19305321375529
ite= 16 total=50 time= 47.16065780321757
ite= 17 total=50 time= 50.10383531252543
ite= 18 total=50 time= 53.03959586620331
ite= 19 total=50 time= 56.00242070754369
ite= 20 total=50 time= 58.94857827822367
ite= 21 total=50 time= 61.89563040335973
ite= 22 total=50 time= 64.83249643643697
ite= 23 total=50 time= 67.78363691965738
ite= 24 total=50